In [1]:
from urllib import request

import tensorflow as tf

from io import StringIO
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from __future__ import print_function
from keras.layers import Dense, Activation, SimpleRNN, LSTM
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
import wordcloud as w
import matplotlib.pyplot as plot
import torch
from transformers import  BartTokenizer, TFBartForConditionalGeneration

2024-11-23 19:40:44.161302: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 19:40:44.171119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732383644.182844  596382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732383644.186387  596382 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 19:40:44.198440: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [279]:
shelley_frank = request.urlopen('https://www.gutenberg.org/cache/epub/84/pg84.txt').read()

In [280]:
shelley_frank = shelley_frank.decode('utf-8-sig')
print(shelley_frank[:100])

The Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus
    
This ebook is for the 


In [281]:
actual_start = 'You will rejoice'
word_index = shelley_frank.find(actual_start)

In [282]:
if word_index != -1:
    shelley_frank = shelley_frank[word_index:]

In [283]:
shelley_frank = re.sub(r'—', ' ', shelley_frank.lower())
shelley_frank = shelley_frank.replace('project gutenberg', '')
shelley_frank = nltk.word_tokenize(shelley_frank)
shelley_frank = ' '.join(shelley_frank)  

# Ensure space after punctuation if it's missing
print(shelley_frank[:100])

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which yo


In [284]:
shelley_frank = re.sub(r'[^a-zA-Z.,!?; ]', '', shelley_frank)
shelley_frank = re.sub(r'([.,!?;])', r'\1 ', shelley_frank)
shelley_frank = re.sub(r'\s+([.,!?;])', r'\1', shelley_frank)
shelley_frank = re.sub(r'\s+', ' ', shelley_frank) 
print(shelley_frank[:1000])

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. i arrived here yesterday, and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking. i am already far north of london, and as i walk in the streets of petersburgh, i feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. do you understand this feeling? this breeze, which has travelled from the regions towards which i am advancing, gives me a foretaste of those icy climes. inspirited by this wind of promise, my daydreams become more fervent and vivid. i try in vain to be persuaded that the pole is the seat of frost and desolation; it ever presents itself to my imagination as the region of beauty and delight. there, margaret, the sun is for ever visible, its broad disk just skirting the horizon and diffusing a perpetual splendour. there for wi

In [285]:
with open('./shelley.txt', 'w') as output_file:
    output_file.write(shelley_frank)

with open('./shelley.txt', 'r') as f:
    text = f.read()

sentences = re.split(r'\.|\n', text)  # This will split by periods and newlines
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

with open('./shelley_processed.txt', 'w') as file:
    for sentence in sentences:
        file.write(sentence + '\n')

# Print the number of lines
#print(len(lines))  # Make sure it prints a value greater than 1

In [286]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

bpe_tokenizer = ByteLevelBPETokenizer()

trainer = trainers.BpeTrainer(vocab_size=20000, min_frequency=2, special_tokens=["<unk>", "<s>", "</s>", "<pad>", "<mask>"])

%time bpe_tokenizer.train(files='./shelley_processed.txt', vocab_size=20000, min_frequency=2)
!mkdir ./transf_shelley
model_name = './transf_shelley'
bpe_tokenizer.save_model(model_name)




CPU times: user 1.55 s, sys: 366 ms, total: 1.92 s
Wall time: 175 ms
mkdir: cannot create directory ‘./transf_shelley’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['./transf_shelley/vocab.json', './transf_shelley/merges.txt']

In [287]:
from transformers import PretrainedConfig

# Create configuration for the tokenizer (for future use)
from transformers import GPT2Config
from transformers import AutoModelForCausalLM

# Define the model configuration for a causal language model
config = GPT2Config(
    vocab_size=20000,                # Set vocab size based on your tokenizer
    n_positions=512,                 # Max sequence length
    n_ctx=512,                       # Context window
    n_embd=768,                      # Embedding size
    n_layer=12,                      # Number of transformer layers
    n_head=12,                       # Number of attention heads
    pad_token_id=50256,              # Default pad token (used in GPT2)
    bos_token_id=50256,              # Beginning of sentence token (GPT2)
    eos_token_id=50256,              # End of sentence token (GPT2)
)

model = AutoModelForCausalLM.from_config(config)
#model.train()

config.save_pretrained(model_name)
# After training, save the model weights and configuration
model.save_pretrained(model_name)

# Save the configuration to the same directory

In [288]:
import torch
use_gpu = -1
if torch.cuda.is_available():
    use_gpu = 0

In [289]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": './shelley_processed.txt'})
datasets = datasets["train"].train_test_split(test_size=0.2)
train_dataset = datasets['train']
eval_dataset = datasets['test']
#model_checkpoint = 'stabilityai/stablelm-zephyr-3b'
model_checkpoint = model_name

Generating train split: 0 examples [00:00, ? examples/s]

In [290]:
print(datasets['train'])

Dataset({
    features: ['text'],
    num_rows: 2474
})


In [291]:
from transformers import AutoTokenizer

w_tokenizer = ByteLevelBPETokenizer(
    './shelley_safe/vocab.json',
    'shelley_safe/merges.txt'
)

# Use the directory where you saved the tokenizer files
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [292]:
def tokenize(examples):
    return tokenizer(examples['text'])

In [293]:
tokenized_data = datasets.map(tokenize, batched = True, num_proc = 1, remove_columns = ['text'])

Map:   0%|          | 0/2474 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

In [294]:
tokenized_data['train'][0]

{'input_ids': [393,
  3453,
  376,
  984,
  1157,
  292,
  3651,
  1415,
  873,
  7648,
  281,
  386,
  451,
  1149,
  421,
  352,
  2357,
  11,
  350,
  259,
  5557,
  343,
  317,
  259,
  524,
  7607,
  2189],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [295]:
# Hyperparams
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_data.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

tokenizer.decode(lm_datasets["train"][1]["input_ids"])

Map:   0%|          | 0/2474 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

' i said, little happiness remains for us on earth; yet all that i may one day enjoy is centred in youthe sweet girl welcomed me with warm affection, yet tears were in her eyes as she beheld my emaciated frame and feverish cheeksyet when she died! nay, then i was not miserablelisten to me, frankensteini passed an hour in this state of mind, when suddenly i reflected how fearful the combat which i momentarily expected would be to my wife, and i earnestly entreated her to retire, resolving not to join her until i had obtained some knowledge as to the situation of my enemyunable to endure the aspect of the being'

In [309]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
from transformers import Trainer, TrainingArguments

#model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-test",
    per_device_train_batch_size=2,  
    gradient_accumulation_steps=4,  
    eval_strategy = "epoch",
    num_train_epochs = 50,
    learning_rate=0.01,
    weight_decay=0.01,
)

In [310]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

In [311]:
% time trainer.train()
trainer.save_model(model_name)

Epoch,Training Loss,Validation Loss
1,No log,6.641420
2,No log,6.509946
3,No log,6.506912
4,No log,6.482923
5,No log,6.447932
6,No log,6.446107
7,No log,6.476734
8,6.331600,6.452150
9,6.331600,6.450957
10,6.331600,6.542045


In [312]:
from transformers import AutoModelForCausalLM, AutoTokenizer

s_model = AutoModelForCausalLM.from_pretrained(model_name)
s_tokenizer = AutoTokenizer.from_pretrained(model_name)


In [315]:
#Txt gen attempr

prompt = 'you will rejoice'

input_ids = s_tokenizer(prompt, return_tensors='pt').input_ids
# model generate with causal lm  = Text generation

outputs = s_model.generate(
    input_ids,
    # Gen text len
    max_length=512,           
    num_return_sequences=1,
    # Sampling should be true if using top_k or top_p word estimeation if false use greedy methods
    do_sample=True,
    #top_k=3,
    top_p=0.95,
    # Set temp for less random gen
    temperature=1.0,
)


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

you will rejoice, for you; a me is from by me of last and was my first be on that i, my power in the sorrows you he was then? own shall soon, butbut he a creature of the moon, and i of a and his friend! the cottagers, be of her my heart the white? it me of a father and my existence to you passed your being, a dungeon and, my departure months a young i said she not and had was with so first the more be the door to the wind a child the words and a thousand, and a smiling he may to his thought and when the sea by the sight was to the nighti had i read other, which i had with an mind world, you to a miserable light of all a young was i saw and how the old when not your father a prison? and the same sun a few candour as a few object thoughts and if and her custom and hatred were a multitude, her names; but, at the wind, she of the night and their mind sky of a feeble from a suffici; the murderer of my thoughts, the arabian been my imagination which at clerval in this place by his heart in h

In [316]:

greedy_search_outputs = s_model.generate(
    input_ids,
    # Gen text len
    max_length=512,           
    num_return_sequences=1,
    temperature=1.0,
)


generated_text = tokenizer.decode(greedy_search_outputs[0], skip_special_tokens=True)
print(generated_text)

you will rejoice, and the same first, i, and the same same of the next i was the same, and was, and the same the same little, i had was the the the own the greatest, and i had was and was i was and, i was the greatest the same, and i was and the same boat and i was i was i had was i was i was and i had was the same the same the the sun, and i was the greatest the father, and the same, and i was and i was the same object, i was and i had the same, and i had been of the same monster and i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was 

In [320]:
beam_search_outputs = s_model.generate(
    input_ids,
    # Gen text len
    max_length=512,           
    num_return_sequences=1,
    num_beams = 2,
    temperature=1.0,
)


generated_text = tokenizer.decode(beam_search_outputs[0], skip_special_tokens=True)
print(generated_text)

you will rejoice, and i was i was i had had not be, and i was i was the world, i had was i, and i was and i had, not be not have been the greatest, and i had was i had was i was i was i was i had been, i had was i had have not be be have been of my father, but, and i was i was i was i had not not have not not me, but, i had have been the whole the same, and was and i had been, and i had was and i was i was and i had not, and i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i was i